# Week 3 Assignemnt: Capstone Project

## PART 1

### 1. Installing Packages 

In [1]:
!conda install -c conda lxml
!pip install et_xmlfile
!pip install bs4
!pip install html5lib
!pip install lxml

import bs4.builder._lxml
from lxml import etree
import requests
from bs4 import BeautifulSoup
import pandas as pd
print('Done all installs')

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - lxml


The following packages will be UPDATED:

    certifi: 2020.4.5.2-py36h9f0ad1d_0 conda-forge --> 2020.4.5.2-py36_0

Preparing transaction: done
Verifying transaction: done
Executing transaction: done
Done all installs


### 2. Scraping Webpage

In [2]:
r = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M')
soup = BeautifulSoup(r.content,features='lxml')

In [3]:
table = soup.find_all('table')[0]
df = pd.read_html(str(table))

In [4]:
toronto_neighbourhoods = pd.DataFrame(df[0])
toronto_neighbourhoods.head()

,Postal Code,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


### 3. Working with Dataframe

In [5]:
toronto_neighbourhoods = toronto_neighbourhoods[~toronto_neighbourhoods.Borough.str.contains("Not assigned")]
toronto_neighbourhoods.head()

,Postal Code,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


In [6]:
toronto_neighbourhoods.shape

(103, 3)

## PART 2

### Reading Coordinates File

In [7]:
temp_df = pd.read_csv('https://cocl.us/Geospatial_data')

In [8]:
temp_df.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


### Finding and Matching Coordinates

In [9]:
long = []
lat = []

for index, row in toronto_neighbourhoods.iterrows():
    for ind, line in temp_df.iterrows():
        if row['Postal Code'] == line['Postal Code']:
            long.append(line['Longitude'])
            lat.append(line['Latitude'])

print("List of Longitudes:")
print(long)
print("")
print("List of Latitudes:")
print(lat)

#toronto_neighbourhoods('Latitude') = lat  
#toronto_neighbourhoods('Longitude') = long  

List of Longitudes:
[-79.3296565, -79.31557159999998, -79.3606359, -79.46476329999999, -79.3894938, -79.53224240000002, -79.19435340000001, -79.352188, -79.309937, -79.37893709999999, -79.44507259999999, -79.55472440000001, -79.16049709999999, -79.340923, -79.3183887, -79.3754179, -79.42819140000002, -79.57720079999999, -79.1887115, -79.2930312, -79.3733064, -79.453512, -79.21691740000001, -79.3634517, -79.3873826, -79.4225637, -79.23947609999999, -79.3634517, -79.4422593, -79.34937190000001, -79.3845675, -79.4422593, -79.23947609999999, -79.3465557, -79.48726190000001, -79.3381065, -79.38175229999999, -79.4197497, -79.26202940000002, -79.385975, -79.46476329999999, -79.352188, -79.38157640000001, -79.42819140000002, -79.2845772, -79.37471409999999, -79.5069436, -79.31557159999998, -79.37981690000001, -79.4900738, -79.56596329999999, -79.23947609999999, -79.40849279999999, -79.49569740000001, -79.340923, -79.4197497, -79.47601329999999, -79.53224240000002, -79.2648481, -79.408492799999

### Appending Coordinates to Dataframe

In [10]:
toronto_neighbourhoods = toronto_neighbourhoods.assign(**{'Longitude': long, 'Latitude': lat,})

In [11]:
toronto_neighbourhoods.head()

,Postal Code,Borough,Neighborhood,Longitude,Latitude
2,M3A,North York,Parkwoods,-79.329656,43.753259
3,M4A,North York,Victoria Village,-79.315572,43.725882
4,M5A,Downtown Toronto,"Regent Park, Harbourfront",-79.360636,43.654260
5,M6A,North York,"Lawrence Manor, Lawrence Heights",-79.464763,43.718518
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",-79.389494,43.662301


## PART 3

In [13]:
import numpy as np # library to handle data in a vectorized manner
!pip install geopy
#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

print("---------------")
print('All libraries imported.')
print("---------------")

---------------
All libraries imported.
---------------


In [14]:
! pip install folium
#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print("---------------")
print("Folium download complete.")
print("---------------")

---------------
Folium download complete.
---------------


In [15]:
print('The dataframe has {} boroughs and {} neighborhoods.'.format(
        len(toronto_neighbourhoods['Borough'].unique()),
        toronto_neighbourhoods.shape[0]
    )
)

The dataframe has 10 boroughs and 103 neighborhoods.


In [16]:
address = 'Toronto, Canada'

geolocator = Nominatim(user_agent="toronto_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.6534817, -79.3839347.


In [17]:
# create map of New York using latitude and longitude values
toronto_map = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(toronto_neighbourhoods['Latitude'], toronto_neighbourhoods['Longitude'], toronto_neighbourhoods['Borough'], toronto_neighbourhoods['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(toronto_map)  
    
toronto_map